In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter

In [ ]:
asteroids_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
asteroids_dir = os.path.join(asteroids_dir, os.listdir(asteroids_dir)[0])
sorted(os.listdir(asteroids_dir))

In [ ]:
list_fits = [im for im in sorted(os.listdir(asteroids_dir)) if ".fits" in im]
fits_sci_image = os.path.join(asteroids_dir, list_fits[0])
fits_sci_image = os.path.abspath(fits_sci_image)
os.path.isfile(fits_sci_image)

In [ ]:
calibration.get_infos_from_image(fits_sci_image)

In [ ]:
bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)
print(bias_dir, darks_dir, flats_dir)

In [ ]:
# Get image directory, failename and extension
sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))
print(sc_im_dir, sc_im_name, sc_im_ext)

In [ ]:
# Get information from FITS header
sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image)

In [ ]:
bias_list = calibration.load_bias_frames(bias_dir, sc_date, sc_cam, sc_x, sc_y, override_date_check=False, max_days=40)
print(bias_list)
MASTER_BIAS = calibration.master_bias(bias_list)

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(MASTER_BIAS["data"], sigma=3.0)

plt.imshow(MASTER_BIAS["data"], cmap="gray", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
# Master dark
# TBD: check if there is already one that works
darks_list = calibration.load_dark_frames(darks_dir, sc_date, sc_cam, sc_expos, sc_x, sc_y)

In [ ]:
MASTER_DARK, HOT_PIXELS = calibration.master_dark(darks_list, master_bias=MASTER_BIAS["path"])

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(MASTER_DARK["data"], sigma=3.0)

plt.imshow(MASTER_DARK["data"], cmap="gray", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
# Master flat
# TBD: check if there is already one that works
flats_list = calibration.load_flat_frames(flats_dir, sc_date, sc_cam, sc_filter, sc_x, sc_y, override_date_check=True)
print(flats_list)

In [ ]:
MASTER_FLAT, DEAD_PIXELS = calibration.master_flat(flats_list)

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(MASTER_FLAT["data"], sigma=3.0)

plt.imshow(MASTER_FLAT["data"], cmap="gray", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
additive_corr = MASTER_DARK["data"] + MASTER_BIAS["data"]
with fits.open(fits_sci_image) as hdul:
    sc_data = hdul[0].data

try:
    RED_SCIENCE = (sc_data - additive_corr) / MASTER_FLAT["data"]
except ValueError:
    RED_SCIENCE = (np.transpose(sc_data) - additive_corr) / MASTER_FLAT["data"]

# Clean bad pixels
# smoothed = median_filter(RED_SCIENCE, size=(5,5))

# Hot pixels
# try:
#    hot_pixel = np.where( HOT_PIXELS["data"]==1 )
#    RED_SCIENCE[hot_pixel] = smoothed[hot_pixel]
# except:
#    print('Cannot clean hot pixels')

# Dead pixels
# try:
#    dead_pixel = np.where( DEAD_PIXELS["data"]==1 )
#    RED_SCIENCE[dead_pixel] = smoothed[dead_pixel]
# except:
#    print('Cannot clean dead pixels')

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(sc_data, sigma=3.0)

plt.imshow(sc_data, cmap="gray", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(RED_SCIENCE, sigma=3.0)

plt.imshow(RED_SCIENCE, cmap="gray", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

# Photometry

In [ ]:
target_type = "variable_star"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
elif target_type == "variable_star":
    data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_VARSTARS)
os.listdir(data_dir)

In [ ]:
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

In [ ]:
achoice = os.listdir(data_dir)[1]
reduced = True
aligned = True
subdata_dir = os.path.join(data_dir, achoice)
if reduced:
    subdata_dir = os.path.join(subdata_dir, "REDUCED")
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in os.path.splitext(im)[-1]]
list_fits = sorted(list_fits)

In [ ]:
list_fits

In [ ]:
rerun = False
use_calib = False
use_sextractor = False
test_mode = False  # Only runs on a few images, prints the tables at each step and does not write files.
sex_photom = False
write_res = True

if rerun or test_mode:
    from tqdm import tqdm
    from astropy.timeseries import TimeSeries
    from astropy.table import Table, vstack
    from astropy.wcs import WCS
    from astropy.wcs.utils import skycoord_to_pixel
    from astropy.coordinates import Angle, SkyCoord
    from astropy.time import Time

    import warnings
    from astropy.utils.exceptions import AstropyWarning
    from astropy.coordinates.name_resolve import NameResolveError

    warnings.simplefilter("ignore", category=AstropyWarning)

    import logging

    logger = logging.getLogger()
    logger.setLevel(logging.CRITICAL)

    if test_mode:
        list_fits = list_fits[:10]
    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(subdata_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            if use_calib:
                bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)

                # Get information from FITS header
                sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=False)
                # print(sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y)

                # Run calibration
                dico_calib = calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, path_to_bias_dir="", use_bias=False, override_date_check=True, max_days=7, speedup=True, verbose=False, write_tmp=True)
                red_sci_image = dico_calib["path"]
            else:
                red_sci_image = fits_sci_image

            # Analysis
            # hdu = fits.open(red_sci_image)[0]
            im_dir = os.path.abspath(os.path.dirname(red_sci_image))
            im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
            with fits.open(red_sci_image) as hdul:
                hdu = hdul[0]
                wcs = WCS(hdu.header)
                epoch = Time(hdu.header.get("DATE-OBS"), format="isot")

                # Get the target position
                target = hdu.header.get("OBJECT")
                if not target_type == "asteroid":
                    try:
                        target_coords = SkyCoord.from_name(target)
                    except NameResolveError:
                        target = target.split("-")[-1]  # Quick fix for the case '330-Adalberta' and similar.
                        target_coords = SkyCoord.from_name(target)
                    target_x, target_y = skycoord_to_pixel(target_coords, wcs=wcs)
                    if test_mode:
                        print(target_x, target_y)
            if loc == 0 or test_mode or sex_photom:
                if use_sextractor:
                    sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME tmp_sources.cat -CATALOG_TYPE FITS_1.0 -GAIN 0.932 -VERBOSE_TYPE QUIET"
                    os.system(sex_cmd)
                    cat_tab = Table.read("tmp_sources.cat")
                    cat_tab.rename_column("X_IMAGE", "xcentroid")
                    cat_tab.rename_column("Y_IMAGE", "ycentroid")
                    _, fwhm, _ = sigma_clipped_stats(cat_tab["FWHM_IMAGE"])
                    sex_coords = SkyCoord(ra=cat_tab["ALPHA_J2000"], dec=cat_tab["DELTA_J2000"], unit="deg", obstime=epoch)
                    # dist_to_target = np.power(cat_tab["xcentroid"]-target_x, 2) + np.power(cat_tab["ycentroid"]-target_y, 2)
                    dist_to_target = sex_coords.separation(target_coords)
                    id_target = np.nanargmin(dist_to_target)
                    if loc == 0:
                        refs_ids = np.argsort(np.abs(cat_tab["FLUX_AUTO"] - cat_tab[id_target]["FLUX_AUTO"]))[1:6]
                        ref_coords = SkyCoord(ra=cat_tab[refs_ids]["ALPHA_J2000"], dec=cat_tab[refs_ids]["DELTA_J2000"], unit="deg", obstime=epoch)
                    else:
                        refs_ids = np.array([np.nanargmin(sex_coords.separation(refradec)) for refradec in ref_coords]).flatten()

                    if sex_photom and not (target_type == "asteroid"):
                        if test_mode:
                            print(id_target, dist_to_target[id_target])
                        # if np.sqrt(dist_to_target[id_target]) < 2*fwhm :
                        sex_target_table = cat_tab[id_target]
                        sex_ref_table = cat_tab[refs_ids]
                        sex_target_ts = TimeSeries(time=[epoch], data=sex_target_table)
                        for i, val in enumerate(sex_ref_table["FLUX_AUTO"]):
                            sex_target_ts.add_column(val, name=f"FLUX_REF{i+1}")
                        if test_mode:
                            print(sex_target_ts)
                        if loc == 0:
                            concat_sex = sex_target_ts
                        else:
                            try:
                                concat_sex = vstack([concat_sex, sex_target_ts])
                            except ValueError:
                                pass
                else:
                    try:
                        sources = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
                        fwhm = photometry.get_fwhm(red_sci_image, sources)
                    except RuntimeError:
                        fwhm = 10
            if target_type == "asteroid":
                phot_target_table = photometry.query_named_sso_photometry(red_sci_image, fwhm, verbose=False)
            elif test_mode or not (sex_photom):
                phot_target_table = photometry.apert_photometry_target(red_sci_image, fwhm, verbose=False)
                ref_target_table = photometry.apert_photometry(red_sci_image, cat_tab[refs_ids], fwhm)
                for i, (ap, bgsub) in enumerate(zip(ref_target_table["aperture_sum"], ref_target_table["aper_sum_bkgsub"])):
                    phot_target_table.add_column(ap, name=f"aperture_sum_REF{i+1}")
                    phot_target_table.add_column(bgsub, name=f"aper_sum_bkgsub_REF{i+1}")
            if loc == 0:
                concat_ts = phot_target_table
            else:
                try:
                    concat_ts = vstack([concat_ts, phot_target_table])
                except ValueError:
                    pass
    if not (test_mode) and sex_photom:
        concat_ts = concat_sex

    if write_res and not (test_mode):
        try:
            if use_calib:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}_CAL.fits", format="fits", overwrite=True)
            else:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}.fits", format="fits", overwrite=True)
        except OSError:
            time.sleep(5)
            if use_calib:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}_CAL.fits", format="fits", overwrite=True)
            else:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}.fits", format="fits", overwrite=True)
    elif test_mode:
        concat_ts.pprint_all()
        concat_sex.pprint_all()
        # plt.errorbar(phot_target_table["Epoch"], phot_target_table["aper_sum_bkgsub"], yerr=phot_target_table["noise"])
else:
    if use_calib:
        if aligned:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits")
        else:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1]}_CAL.fits", format="fits")
    else:
        if aligned:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits")
        else:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1]}.fits", format="fits")

In [ ]:
from astropy.stats import sigma_clipped_stats


def detrend_fun(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d


from scipy.optimize import curve_fit

if sex_photom:
    mean, med, sig = sigma_clipped_stats(concat_ts["FLUX_AUTO"])

    sel = np.logical_and(concat_ts["FLUX_AUTO"] > med - 7 * sig, concat_ts["FLUX_AUTO"] < med + 7 * sig)
    plt.errorbar(concat_ts[sel].time.mjd, concat_ts[sel]["FLUX_AUTO"] / med.value, yerr=concat_ts[sel]["FLUXERR_AUTO"] / med.value, fmt=".", c="k")
    for i in range(1, 6):
        mean, med, sig = sigma_clipped_stats(concat_ts[f"FLUX_REF{i}"])
        sel = np.logical_and(concat_ts[f"FLUX_REF{i}"] > med - 7 * sig, concat_ts[f"FLUX_REF{i}"] < med + 7 * sig)
        plt.scatter(concat_ts[sel].time.datetime64, concat_ts[sel][f"FLUX_REF{i}"] / med, marker="+", s=2)
        plt.ylim(0.8, 1.2)
    detrend_coeffs = curve_fit(detrend_fun, concat_ts[sel].time.mjd, concat_ts[sel]["FLUX_AUTO"], sigma=concat_ts[sel]["FLUXERR_AUTO"])[0]
    plt.plot(concat_ts[sel].time.mjd, detrend_fun(concat_ts[sel].time.mjd, *detrend_coeffs) / med.value)
else:
    mean, med, sig = sigma_clipped_stats(concat_ts["aper_sum_bkgsub"])
    sel = np.logical_and(concat_ts["aper_sum_bkgsub"] > med - 3 * sig, concat_ts["aper_sum_bkgsub"] < med + 3 * sig)
    plt.scatter(concat_ts[sel].time.mjd, concat_ts[sel]["aper_sum_bkgsub"] / med)
    plt.tick_params(axis="x", labelrotation=90)
    detrend_coeffs = curve_fit(detrend_fun, concat_ts[sel].time.mjd, concat_ts[sel]["aper_sum_bkgsub"].value)[0]
    plt.plot(concat_ts[sel].time.mjd, detrend_fun(concat_ts[sel].time.mjd, *detrend_coeffs) / med)

In [ ]:
detrend = detrend_fun(concat_ts[sel].time.mjd, *detrend_coeffs)
plt.errorbar(concat_ts[sel].time.mjd, concat_ts[sel]["aper_sum_bkgsub"] / detrend, yerr=concat_ts[sel]["noise"] / detrend, fmt=".")

In [ ]:
def sin_to_fit(t, A, nu, phi, C):
    return A * np.sin(nu * t + phi) + C


signal = concat_ts[sel]["aper_sum_bkgsub"] / detrend
noise = concat_ts[sel]["noise"] / detrend


sin_coeffs = curve_fit(sin_to_fit, concat_ts[sel].time.mjd, signal, sigma=noise)[0]
plt.errorbar(concat_ts[sel].time.mjd, signal, yerr=noise, fmt=".")
plt.plot(concat_ts[sel].time.mjd, sin_to_fit(concat_ts[sel].time.mjd, *sin_coeffs))

In [ ]:
def d_sur_dt(t, val):
    return np.diff(val) / np.diff(t)


plt.plot(concat_ts[sel].time.mjd[:-1], d_sur_dt(concat_ts[sel].time.mjd, signal))

In [ ]:
mean = np.mean(signal)
sig = np.std(signal)
plt.errorbar(concat_ts[sel].time.mjd, signal, yerr=noise, fmt=".")
plt.axhline(mean)
plt.fill_between(concat_ts[sel].time.mjd, mean - sig, mean + sig, alpha=0.5)

In [ ]:
t_cut = np.logical_and(concat_ts[sel].time.mjd - 5.9493e4 > 0.037, concat_ts[sel].time.mjd - 5.9493e4 < 0.082)
plt.errorbar(concat_ts[sel].time.mjd[t_cut], signal[t_cut], yerr=noise[t_cut], fmt=".")

In [ ]:
time = concat_ts[sel].time.mjd - 5.9494e4
t_cut = np.logical_and(time > 0.037, time < 0.082)
vmax, vmin = np.max(signal[t_cut]), np.min(signal[t_cut])
vmoy = np.mean(signal[t_cut])


def sin_to_fit(t, nu, phi):
    return vmoy + np.sin((2 * np.pi / nu) * t + phi) * 0.5 * (vmax - vmin)


sin_coeffs = curve_fit(sin_to_fit, time[t_cut], signal[t_cut], sigma=noise[t_cut])[0]
plt.errorbar(time[t_cut], signal[t_cut], yerr=noise[t_cut], fmt=".")
plt.plot(time[t_cut], sin_to_fit(time[t_cut], *sin_coeffs))

In [ ]:
2 * sin_coeffs[0] * 24

In [ ]:
3.56 / 24 / 2